In [99]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
import os
from os.path import isfile, isdir, join
import ntpath

import pandas as pd
from random import shuffle

import cv2

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

## Params

In [100]:
# SELECT EXPERIMENT SAMPLE
#sample_path = 'experiment-samples/trivial'
#sample_path = 'experiment-samples/extra-small'
#sample_path = 'experiment-samples/small-pabk'
#sample_path = 'experiment-samples-nogit/medium'
#sample_path = 'experiment-samples-nogit/large-pabk'
sample_path = 'experiment-samples-nogit/large-csob'


trainSampleGroupSize = 1
testSampleGroupSize = 100

IMAGE_WIDTH = 40
IMAGE_HEIGHT = 60

LR = 0.0001

MODEL_NAME = 'prvy pokus'

## Script

In [101]:
# LOAD LIST OF AVAILABLE FILES
recordings_path = sample_path + '/recordings'
recordings = [d for d in listdir(recordings_path) if isdir(join(recordings_path, d))]
paths = []
for recording in recordings:
    recording_path = recordings_path + '/' + recording
    paths = paths + [(recording_path+'/'+f) for f in listdir(recording_path) if isfile(join(recording_path, f))]
    
# LOAD CORRESPONDING LABELS
labels = pd.read_excel(sample_path+'/labels.xlsx', sheetname='labels')
labels.scene = pd.to_numeric(labels.scene.replace('none','0'))
labels = labels[labels.frame.isin([ntpath.basename(path) for path in paths])]

# SAMPLE FRAMES (TODO poriadne)
train_table = labels.groupby('scene', group_keys=False).apply(lambda x: x.sample(min(len(x), trainSampleGroupSize)))
test_table = labels.groupby('scene', group_keys=False).apply(lambda x: x.sample(min(len(x), testSampleGroupSize)))

In [102]:
unique_labels = labels.scene.unique()

def create_label(scene):
    res = []
    for label in unique_labels:
        if (label == scene):
            res.append(1)
        else:
            res.append(0)
    return np.array(res)

def create_data(df):
    data = []
    for i, row in df.iterrows():
        path = recordings_path+'/'+row.recording+'/'+row.frame
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (IMAGE_WIDTH, IMAGE_HEIGHT))
        data.append([np.array(img_data), create_label(row.scene)])
    shuffle(data)
    return data

train = create_data(train_table)
test = create_data(test_table)

# pootacat polia
X_train = np.array([i[0] for i in train]).reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, 1)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, 1)
y_test = [i[1] for i in test]

In [103]:
tf.reset_default_graph()
convnet = input_data(shape=[None, IMAGE_WIDTH, IMAGE_HEIGHT, 1], name='input')
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = fully_connected(convnet, 512, activation ='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 6, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy')

In [104]:
model = tflearn.DNN(convnet, tensorboard_dir='log-final/csob-train1-2nd', tensorboard_verbose=0)

In [106]:
model.fit (X_train, 
           y_train, 
           n_epoch=200, 
           validation_set=(X_test, y_test),
           snapshot_step = 500,
           show_metric = True,
           run_id = MODEL_NAME)

---------------------------------
Run id: prvy pokus
Log directory: log-final/csob-train1-2nd/
---------------------------------
Training samples: 6
Validation samples: 600
--
Training Step: 1001  | total loss: 0.18880 | time: 1.893s
| Adam | epoch: 1001 | loss: 0.18880 - acc: 0.9724 | val_loss: 3.50182 - val_acc: 0.2683 -- iter: 6/6
--
Training Step: 1002  | total loss: 0.18282 | time: 1.222s
| Adam | epoch: 1002 | loss: 0.18282 - acc: 0.9751 | val_loss: 3.57630 - val_acc: 0.2650 -- iter: 6/6
--
Training Step: 1003  | total loss: 0.17461 | time: 1.052s
| Adam | epoch: 1003 | loss: 0.17461 - acc: 0.9776 | val_loss: 3.57584 - val_acc: 0.2683 -- iter: 6/6
--
Training Step: 1004  | total loss: 0.16830 | time: 1.042s
| Adam | epoch: 1004 | loss: 0.16830 - acc: 0.9799 | val_loss: 3.58894 - val_acc: 0.2683 -- iter: 6/6
--
Training Step: 1005  | total loss: 0.16297 | time: 1.633s
| Adam | epoch: 1005 | loss: 0.16297 - acc: 0.9819 | val_loss: 3.69524 - val_acc: 0.2633 -- iter: 6/6
--
Training 

Training Step: 1046  | total loss: 0.47002 | time: 1.336s
| Adam | epoch: 1046 | loss: 0.47002 - acc: 0.9216 | val_loss: 3.60617 - val_acc: 0.2683 -- iter: 6/6
--
Training Step: 1047  | total loss: 0.43215 | time: 1.046s
| Adam | epoch: 1047 | loss: 0.43215 - acc: 0.9294 | val_loss: 3.63952 - val_acc: 0.2667 -- iter: 6/6
--
Training Step: 1048  | total loss: 0.39768 | time: 1.038s
| Adam | epoch: 1048 | loss: 0.39768 - acc: 0.9365 | val_loss: 3.76255 - val_acc: 0.2217 -- iter: 6/6
--
Training Step: 1049  | total loss: 0.36343 | time: 1.643s
| Adam | epoch: 1049 | loss: 0.36343 - acc: 0.9428 | val_loss: 3.76589 - val_acc: 0.2217 -- iter: 6/6
--
Training Step: 1050  | total loss: 0.33789 | time: 1.262s
| Adam | epoch: 1050 | loss: 0.33789 - acc: 0.9485 | val_loss: 3.65973 - val_acc: 0.2650 -- iter: 6/6
--
Training Step: 1051  | total loss: 0.31256 | time: 1.053s
| Adam | epoch: 1051 | loss: 0.31256 - acc: 0.9537 | val_loss: 3.58624 - val_acc: 0.2417 -- iter: 6/6
--
Training Step: 1052  |

Training Step: 1092  | total loss: 0.08258 | time: 1.044s
| Adam | epoch: 1092 | loss: 0.08258 - acc: 0.9953 | val_loss: 3.89673 - val_acc: 0.2200 -- iter: 6/6
--
Training Step: 1093  | total loss: 0.08132 | time: 1.526s
| Adam | epoch: 1093 | loss: 0.08132 - acc: 0.9957 | val_loss: 3.77370 - val_acc: 0.2667 -- iter: 6/6
--
Training Step: 1094  | total loss: 0.07785 | time: 1.198s
| Adam | epoch: 1094 | loss: 0.07785 - acc: 0.9962 | val_loss: 3.71729 - val_acc: 0.2683 -- iter: 6/6
--
Training Step: 1095  | total loss: 0.07424 | time: 1.045s
| Adam | epoch: 1095 | loss: 0.07424 - acc: 0.9966 | val_loss: 3.77741 - val_acc: 0.2667 -- iter: 6/6
--
Training Step: 1096  | total loss: 0.07429 | time: 1.037s
| Adam | epoch: 1096 | loss: 0.07429 - acc: 0.9969 | val_loss: 3.97007 - val_acc: 0.2200 -- iter: 6/6
--
Training Step: 1097  | total loss: 0.07066 | time: 1.534s
| Adam | epoch: 1097 | loss: 0.07066 - acc: 0.9972 | val_loss: 4.01811 - val_acc: 0.2183 -- iter: 6/6
--
Training Step: 1098  |

Training Step: 1138  | total loss: 0.29386 | time: 1.043s
| Adam | epoch: 1138 | loss: 0.29386 - acc: 0.9663 | val_loss: 3.66667 - val_acc: 0.2317 -- iter: 6/6
--
Training Step: 1139  | total loss: 0.26799 | time: 1.041s
| Adam | epoch: 1139 | loss: 0.26799 - acc: 0.9697 | val_loss: 3.73069 - val_acc: 0.2283 -- iter: 6/6
--
Training Step: 1140  | total loss: 0.25257 | time: 1.615s
| Adam | epoch: 1140 | loss: 0.25257 - acc: 0.9727 | val_loss: 3.64946 - val_acc: 0.2383 -- iter: 6/6
--
Training Step: 1141  | total loss: 0.24065 | time: 1.146s
| Adam | epoch: 1141 | loss: 0.24065 - acc: 0.9754 | val_loss: 3.46414 - val_acc: 0.2700 -- iter: 6/6
--
Training Step: 1142  | total loss: 0.22235 | time: 1.044s
| Adam | epoch: 1142 | loss: 0.22235 - acc: 0.9779 | val_loss: 3.42699 - val_acc: 0.2483 -- iter: 6/6
--
Training Step: 1143  | total loss: 0.20377 | time: 1.040s
| Adam | epoch: 1143 | loss: 0.20377 - acc: 0.9801 | val_loss: 3.50172 - val_acc: 0.2700 -- iter: 6/6
--
Training Step: 1144  |

Training Step: 1184  | total loss: 0.67737 | time: 1.572s
| Adam | epoch: 1184 | loss: 0.67737 - acc: 0.8900 | val_loss: 3.23525 - val_acc: 0.2383 -- iter: 6/6
--
Training Step: 1185  | total loss: 0.61339 | time: 1.156s
| Adam | epoch: 1185 | loss: 0.61339 - acc: 0.9010 | val_loss: 3.18034 - val_acc: 0.2367 -- iter: 6/6
--
Training Step: 1186  | total loss: 0.55927 | time: 1.043s
| Adam | epoch: 1186 | loss: 0.55927 - acc: 0.9109 | val_loss: 3.20500 - val_acc: 0.2383 -- iter: 6/6
--
Training Step: 1187  | total loss: 0.51545 | time: 1.473s
| Adam | epoch: 1187 | loss: 0.51545 - acc: 0.9198 | val_loss: 3.32180 - val_acc: 0.2667 -- iter: 6/6
--
Training Step: 1188  | total loss: 0.46934 | time: 1.252s
| Adam | epoch: 1188 | loss: 0.46934 - acc: 0.9278 | val_loss: 3.40888 - val_acc: 0.2383 -- iter: 6/6
--
Training Step: 1189  | total loss: 0.42632 | time: 1.047s
| Adam | epoch: 1189 | loss: 0.42632 - acc: 0.9350 | val_loss: 3.38953 - val_acc: 0.2550 -- iter: 6/6
--
Training Step: 1190  |